In [1]:
#import necessary tools
import re
from string import digits
import pandas as pd
import numpy as np

In [17]:
#read the documents
with open('101.txt','rb') as f:
    contents = f.read()
    contents = contents.decode("utf-16").rstrip("\n")
    contents = contents.split("\r\n") #maybe need to strip \t?

In [20]:
BlockNum=[]
Procedure=list()
TrialNUM =[1] #somehow eprime writes as TrialNUM
img1 =[]
img2=[]
Choice_RESP = []
Choice_RT=[]


#loop through each line
for line in range(len(contents)-20):
    
    #this line of "running: listlike/black" is in the fixed position of each of 4 conditions (b1b2l1l2)
    #therefore, we use it as our anchor to locate meaningful blocks
    head = contents[line+9]
    if contents[line] == '\t*** LogFrame Start ***' and (head.startswith("\tRunning: ListLike") == True or head.startswith("\tRunning: ListBlack") == True):
        
        #loop inside the meaningful block                
        for inner_line in range(20):
            
            new_line = line + inner_line
            
            #mark block type
            if contents[new_line].startswith("\tProcedure: "):
                Procedurea = contents[new_line].split("Procedure: ",1)[1] 
                Procedure.append(Procedurea)

            #mark img1
            if contents[new_line].startswith("\timg1: "):
                img1a = contents[new_line].split("img1: ",1)[1] 
                img1.append(img1a)
            
            #mark img2
            if contents[new_line].startswith("\timg2: "):
                img2a = contents[new_line].split("img2: ",1)[1] 
                img2.append(img2a)
                
            #mark block number
            if contents[new_line].startswith("\tBlockNum: "):
                BlockNuma = contents[new_line].split("BlockNum: ",1)[1]
                BlockNuma =[''.join(c for c in BlockNuma if c in digits)]
                BlockNum=BlockNum+BlockNuma
                
            #mark trial number    -> maybe some calculation? such as 50* if ends in 2, or just plus one
            if contents[new_line].startswith("\tTrialNUM: "):
                #or just +=1
                TrialNUMa = TrialNUM[-1] + 1
                
                #TrialNUMa = contents[new_line].split("TrialNUM: ",1)[1]
                #TrialNUMa =[''.join(c for c in TrialNUMa if c in digits)]
                #TrialNUMa = list([float(i) for i in TrialNUMa])
                TrialNUM.append(TrialNUMa)
            
            
            #mark choice Black response time
            if contents[new_line].startswith("\tChoiceB.RT: "):
                Choice_RESPa = contents[new_line].split("ChoiceB.RT: ",1)[1]
                Choice_RESPa =[''.join(c for c in Choice_RESPa if c in digits)]
                Choice_RESP=Choice_RESP+Choice_RESPa
                
            #mark choice Like response time
            if contents[new_line].startswith("\tChoiceL.RT: "):
                Choice_RESPa = contents[new_line].split("ChoiceL.RT: ",1)[1]
                Choice_RESPa =[''.join(c for c in Choice_RESPa if c in digits)]
                Choice_RESP=Choice_RESP+Choice_RESPa
                
            #mark choice Black response  
            if contents[new_line].startswith("\tChoiceB.RESP: ") :
                Choice_RTa = contents[new_line].split("ChoiceB.RESP: ",1)[1]
                Choice_RTa =[''.join(c for c in Choice_RTa if c in digits)]
                Choice_RTa = list([float(i) for i in Choice_RTa])
                Choice_RT=Choice_RT+Choice_RTa
                
            #mark choice Like response 
            if contents[new_line].startswith("\tChoiceL.RESP: "):
                Choice_RTa = contents[new_line].split("ChoiceL.RESP: ",1)[1]
                Choice_RTa =[''.join(c for c in Choice_RTa if c in digits)]
                Choice_RTa = list([float(i) for i in Choice_RTa])
                Choice_RT=Choice_RT+Choice_RTa


#Assign variable names
Procedure_df = pd.DataFrame(Procedure)
Procedure_df.columns = ['Btype']  
        
img1 = [int(i) for i in img1] 
img1_df = pd.DataFrame(img1)
img1_df.columns = ['img1']

img2 = [int(i) for i in img2] 
img2_df = pd.DataFrame(img2)
img2_df.columns = ['img2']

TrialNUM = [int(i) for i in TrialNUM] 
TrialNUM_df = pd.DataFrame(TrialNUM)
TrialNUM_df.columns = ['trial']

BlockNum = [int(i) for i in BlockNum] 
BlockNum_df = pd.DataFrame(BlockNum)
BlockNum_df.columns = ['block']

Choice_RESP = [int(i) for i in Choice_RESP] 
Choice_RESP_df = pd.DataFrame(Choice_RESP)
Choice_RESP_df.columns = ['RESP']

Choice_RT = [int(i) for i in Choice_RT] 
Choice_RT_df = pd.DataFrame(Choice_RT)
Choice_RT_df.columns = ['RT']


In [19]:
#merge the data
sub_id = contents[16].split("Subject: ",1)[1] 
sub_df = pd.DataFrame(np.repeat(sub_id, img1_df.shape[0], axis = 0))
sub_df.columns = ['sub']

all = pd.concat([sub_df.reset_index(drop=True), BlockNum_df], axis=1)
all = pd.concat([all.reset_index(drop=True), Procedure_df], axis=1)
all = pd.concat([all.reset_index(drop=True), TrialNUM_df], axis=1)
all = pd.concat([all.reset_index(drop=True), img1_df], axis=1)
all = pd.concat([all.reset_index(drop=True), img2_df], axis=1)
all = pd.concat([all.reset_index(drop=True), Choice_RESP_df], axis=1)
all = pd.concat([all.reset_index(drop=True), Choice_RT_df], axis=1)

all

,sub,block,Btype,trial,img1,img2,RESP,RT
0,101,1.0,ChooseBlack,1,189.0,191.0,1118.0,1.0
1,101,1.0,ChooseBlack,2,142.0,187.0,949.0,2.0
2,101,1.0,ChooseBlack,3,129.0,151.0,963.0,1.0
3,101,1.0,ChooseBlack,4,150.0,193.0,879.0,1.0
4,101,1.0,ChooseBlack,5,177.0,178.0,742.0,1.0
...,...,...,...,...,...,...,...,...
195,101,4.0,ChooseLike,196,189.0,217.0,3279.0,1.0
196,101,4.0,ChooseLike,197,176.0,217.0,2106.0,2.0
197,101,4.0,ChooseLike,198,133.0,150.0,3696.0,2.0
198,101,4.0,ChooseLike,199,172.0,206.0,1410.0,2.0


In [11]:
all.to_excel("f_209_1.xlsx")